# <B> Setup for SageMaker Immersion Day with DL Framework </B>
* Container: conda_python3

## AutoReload

In [2]:
%load_ext autoreload
%autoreload 2

## 0. Install packages

In [2]:
install_needed = True  # should only be True once
# install_needed = False

In [3]:
%%bash
#!/bin/bash

DAEMON_PATH="/etc/docker"
MEMORY_SIZE=10G

FLAG=$(cat $DAEMON_PATH/daemon.json | jq 'has("data-root")')
# echo $FLAG

if [ "$FLAG" == true ]; then
    echo "Already revised"
else
    echo "Add data-root and default-shm-size=$MEMORY_SIZE"
    sudo cp $DAEMON_PATH/daemon.json $DAEMON_PATH/daemon.json.bak
    sudo cat $DAEMON_PATH/daemon.json.bak | jq '. += {"data-root":"/home/ec2-user/SageMaker/.container/docker","default-shm-size":"'$MEMORY_SIZE'"}' | sudo tee $DAEMON_PATH/daemon.json > /dev/null
    sudo service docker restart
    echo "Docker Restart"
fi

Already revised


In [4]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U smdebug sagemaker-experiments
    !{sys.executable} -m pip install -U sagemaker
    
    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 786.8/786.8 kB 21.7 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for sagemaker: filename=sagemaker-2.154.0-py2.py3-none-any.whl size=1062840 sha256=3428367cb71335e00556483afe871b52f318e680a2e206b2ef685b46910e589b
  Stored in directory: /home/ec2-user/.cache/pip/wheels/9c/74/e3/dc8373ea621e1a5f92e8512c6e1037cceb9f84da169255700b
Successfully built sagemaker
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.152.0
    Uninstalling sagemaker-2.152.0:
      Successfully uninstalled sagemaker-2.152.0


## 1. Set roles

In [3]:
from sagemaker import get_execution_role

In [4]:
strSageMakerRoleName = get_execution_role().rsplit('/', 1)[-1]
print (f"SageMaker Execution Role Name: {strSageMakerRoleName}")

SageMaker Execution Role Name: AmazonSageMaker-ExecutionRole-20221206T163436


### 1.1 Attach IAM polich to sagemaker execution role (<b>with console</b>)
> step 1. IAM console 로 이동    
>
> step 2. 왼쪽 네비게이터에서  "Role" 선택  
> ![nn](images/Role.png)  
>
> step 3. SageMaker Execution Role 검색 후 role 선택 (상위 cell output 참조)  
> ![nn](images/search-by-rolename.png)  
>
> step 4. "attach policies" 메뉴 선택 
> ![nn](images/attach-policy-menu.png)  
>
> step 5. "IAMFullAccess" policy 검색 후 attach 
> ![nn](images/attach-policy.png) 

In [5]:
from utils.iam import iam_handler

In [6]:
iam = iam_handler()

In [7]:
strID = "DJ"
strPrefix = f'{strID}-SM-PIPELINE'

In [8]:
strPrefix

'DJ-SM-PIPELINE'

### 1.2 Attach policy - sagemaker execution role
> SSM  
> S3 <BR>
> CodeCommit <BR>
> CodeBuild <BR>
> CodePipeline <BR>
> EventBridge <BR>
> Lambda <BR>
> ECR

In [9]:
listPolicyArn=[
    "arn:aws:iam::aws:policy/AmazonSSMFullAccess",
    #"arn:aws:iam::aws:policy/AmazonS3FullAccess",
    "arn:aws:iam::aws:policy/AWSCodeCommitFullAccess",
    "arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess",
    "arn:aws:iam::aws:policy/AWSCodePipeline_FullAccess",
    "arn:aws:iam::aws:policy/AmazonEventBridgeFullAccess",
    "arn:aws:iam::aws:policy/AWSLambda_FullAccess",
    "arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryFullAccess"
]
for strPolicyArn in listPolicyArn: iam.attach_policy(strSageMakerRoleName, strPolicyArn)

### 1.3 Create role for lambda

In [10]:
listService = ["lambda.amazonaws.com"]
strRoleName = "-".join([strPrefix, "LabmdaRole"])
listPolicyArn = [
    "arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole",
    "arn:aws:iam::aws:policy/AWSLambda_FullAccess",
    "arn:aws:iam::aws:policy/AmazonSageMakerFullAccess",
]
strDescription="Role for Lambda to call SageMaker functions'"

In [11]:
lambdaRoleArn = iam.create_role(
    listService=listService,
    strRoleName=strRoleName,
    listPolicyArn = listPolicyArn,
    strDescription=strDescription
)

== CREATE ROLE ==
  Role Name: [DJ-SM-PIPELINE-LabmdaRole] is already exist!!, so, this will be deleted and re-created.
{'ResponseMetadata': {'RequestId': 'de23e205-7a0c-497a-a313-936d8c0c38e3', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'de23e205-7a0c-497a-a313-936d8c0c38e3', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Fri, 12 May 2023 00:27:02 GMT'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole
{'ResponseMetadata': {'RequestId': '2eaccd8f-75e8-44c7-ac3a-bdc62b199120', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '2eaccd8f-75e8-44c7-ac3a-bdc62b199120', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Fri, 12 May 2023 00:27:02 GMT'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/AmazonSageMakerFullAccess
{'ResponseMetadata': {'RequestId': '93023317-90ac-420d-b98c-12b75a564969', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '93023317-90ac-420d-b98c-12b75a564969', 'content-ty

### 1.4 role-codebuild

In [12]:
listService = ["codebuild.amazonaws.com", "sagemaker.amazonaws.com"]
strRoleName = "-".join([strPrefix, "CodeBuildRole"])
listPolicyArn = [
    "arn:aws:iam::aws:policy/AWSCodeCommitFullAccess",
    "arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess",
    "arn:aws:iam::aws:policy/AmazonS3FullAccess",
    "arn:aws:iam::aws:policy/CloudWatchFullAccess",
    "arn:aws:iam::aws:policy/AmazonSageMakerFullAccess",
    "arn:aws:iam::aws:policy/AWSLambda_FullAccess",
    "arn:aws:iam::aws:policy/AmazonSSMFullAccess",
]
strDescription="Role for CodeBuild"

In [13]:
buildRoleArn = iam.create_role(
    listService=listService,
    strRoleName=strRoleName,
    listPolicyArn = listPolicyArn,
    strDescription=strDescription
)
print (f"CODE-BUILD-ROLE-ARN: {buildRoleArn}")

== CREATE ROLE ==
  Role Name: [DJ-SM-PIPELINE-CodeBuildRole] is already exist!!, so, this will be deleted and re-created.
{'ResponseMetadata': {'RequestId': 'd365f3b0-6e91-4670-a732-4782439d448d', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'd365f3b0-6e91-4670-a732-4782439d448d', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Fri, 12 May 2023 00:27:03 GMT'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/AmazonSSMFullAccess
{'ResponseMetadata': {'RequestId': '964e02a9-fee6-4ab5-9413-9a177ee932a7', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '964e02a9-fee6-4ab5-9413-9a177ee932a7', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Fri, 12 May 2023 00:27:03 GMT'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess
{'ResponseMetadata': {'RequestId': '15161ec4-bdd8-44c6-9859-71852b7f3b91', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '15161ec4-bdd8-44c6-9859-71852b7f3b91', 'content-type': 'text/xml', 'co

### 1.5 role-codepipeline

In [14]:
listService = ["codepipeline.amazonaws.com", "codebuild.amazonaws.com"]
strRoleName = "-".join([strPrefix, "CodePipelineRole"])
listPolicyArn = [
    "arn:aws:iam::aws:policy/AWSCodeCommitFullAccess",
    "arn:aws:iam::aws:policy/AmazonS3FullAccess",
    "arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess",
    "arn:aws:iam::aws:policy/AWSCodeDeployFullAccess",
    "arn:aws:iam::aws:policy/AWSCodePipeline_FullAccess"
]
strDescription="Role for CodePipeline"

In [15]:
pipelineRoleArn = iam.create_role(
    listService=listService,
    strRoleName=strRoleName,
    listPolicyArn = listPolicyArn,
    strDescription=strDescription
)

print (f"CODE-PIPELINE-ROLE-ARN: {pipelineRoleArn}")

== CREATE ROLE ==
  Role Name: [DJ-SM-PIPELINE-CodePipelineRole] is already exist!!, so, this will be deleted and re-created.
{'ResponseMetadata': {'RequestId': '5fcbff43-2a75-47ed-baf2-d6ddbc8096cb', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '5fcbff43-2a75-47ed-baf2-d6ddbc8096cb', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Fri, 12 May 2023 00:27:05 GMT'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess
{'ResponseMetadata': {'RequestId': '82e43d60-70f3-4dd5-833f-b30e3261aac8', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '82e43d60-70f3-4dd5-833f-b30e3261aac8', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Fri, 12 May 2023 00:27:05 GMT'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/AWSCodeCommitFullAccess
{'ResponseMetadata': {'RequestId': '1c3e8a4a-2fd0-4719-9327-1ef97f0cf963', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '1c3e8a4a-2fd0-4719-9327-1ef97f0cf963', 'content-type': 'text/xm

### 1.6 role-eventbridge

In [16]:
listService = ["events.amazonaws.com"]
strRoleName = "-".join([strPrefix, "EventBridgeRole"])
listPolicyArn = [
    "arn:aws:iam::aws:policy/AmazonEventBridgeFullAccess",
]
strDescription="Role for EventBridge"

In [17]:
eventBridgeRoleArn = iam.create_role(
    listService=listService,
    strRoleName=strRoleName,
    listPolicyArn = listPolicyArn,
    strDescription=strDescription
)

print (f"CODE-PIPELINE-ROLE-ARN: {eventBridgeRoleArn}")

== CREATE ROLE ==
  Role Name: [DJ-SM-PIPELINE-EventBridgeRole] is already exist!!, so, this will be deleted and re-created.
{'ResponseMetadata': {'RequestId': '2fc1967a-9804-45c6-9dd3-edfca7947627', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '2fc1967a-9804-45c6-9dd3-edfca7947627', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Fri, 12 May 2023 00:27:06 GMT'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/AmazonEventBridgeFullAccess
  Service name: ['events.amazonaws.com'], 
  Role name: DJ-SM-PIPELINE-EventBridgeRole, 
  Policys: ['arn:aws:iam::aws:policy/AmazonEventBridgeFullAccess'], 
  Role ARN: arn:aws:iam::419974056037:role/DJ-SM-PIPELINE-EventBridgeRole
== COMPLETED ==
CODE-PIPELINE-ROLE-ARN: arn:aws:iam::419974056037:role/DJ-SM-PIPELINE-EventBridgeRole


## 2. Set datasets and utils

In [18]:
import os
import boto3
import sagemaker
import pandas as pd
from utils.s3 import s3_handler
from utils.ssm import parameter_store

### 2.1 download and manipulate data

In [19]:
s3 = boto3.client("s3")

In [20]:
strDataPath = os.path.join("./dataset", "abalone.csv")
s3.download_file("sagemaker-sample-files", f"datasets/tabular/uci_abalone/abalone.csv", strDataPath)

listFeatureName = [
    "Sex",
    "Length",
    "Diameter",
    "Height",
    "Whole weight",
    "Shucked weight",
    "Viscera weight",
    "Shell weight",
    "Rings",
]
pdData = pd.read_csv(strDataPath, header=None, names=listFeatureName)

# SageMaker XGBoost has the convention of label in the first column
pdData = pdData[listFeatureName[-1:] + listFeatureName[:-1]]
pdData["Sex"] = pdData["Sex"].astype("category").cat.codes
pdData.to_csv(strDataPath, header=True, index=False)

### 2.2 upload to S3 and regiter path to parameter store

In [21]:
strRegionName=boto3.Session().region_name
s3 = s3_handler()
pm = parameter_store(strRegionName)

This is a S3 handler with [None] region.


In [22]:
strBucketName = sagemaker.Session().default_bucket()

In [23]:
source_dir, target_bucket, target_dir = os.path.dirname(strDataPath), strBucketName, "-".join([strPrefix, "DATA"])
s3.upload_dir(source_dir, target_bucket, target_dir)
pm.put_params(key="-".join([strPrefix, "DATA-PATH"]), value=f"s3://{target_bucket}/{target_dir}", overwrite=True)

Upload:[./dataset] was uploaded to [s3://sagemaker-us-east-1-419974056037/DJ-SM-PIPELINE-DATA]successfully


'Store suceess'

### 2.3 Copy utils to pipeline/utils

In [24]:
from distutils.dir_util import copy_tree

In [25]:
copy_tree("./utils", "./mlops/pipeline/utils")

['./mlops/pipeline/utils/lambda_func.py',
 './mlops/pipeline/utils/monitoringjob_utils.py',
 './mlops/pipeline/utils/ssm.py',
 './mlops/pipeline/utils/s3.py',
 './mlops/pipeline/utils/ecr.py',
 './mlops/pipeline/utils/iam.py',
 './mlops/pipeline/utils/.ipynb_checkpoints/s3-checkpoint.py',
 './mlops/pipeline/utils/.ipynb_checkpoints/iam-checkpoint.py',
 './mlops/pipeline/utils/.ipynb_checkpoints/lambda_func-checkpoint.py',
 './mlops/pipeline/utils/.ipynb_checkpoints/ssm-checkpoint.py',
 './mlops/pipeline/utils/.ipynb_checkpoints/monitoringjob_utils-checkpoint.py',
 './mlops/pipeline/utils/__pycache__/ssm.cpython-39.pyc',
 './mlops/pipeline/utils/__pycache__/ecr.cpython-310.pyc',
 './mlops/pipeline/utils/__pycache__/s3.cpython-310.pyc',
 './mlops/pipeline/utils/__pycache__/iam.cpython-310.pyc',
 './mlops/pipeline/utils/__pycache__/ssm.cpython-37.pyc',
 './mlops/pipeline/utils/__pycache__/ssm.cpython-310.pyc',
 './mlops/pipeline/utils/__pycache__/monitoringjob_utils.cpython-310.pyc',
 './

## 3. Create custom docker image

In [38]:
from utils.ecr import ecr_handler
ecr = ecr_handler()
strAccountID = boto3.client("sts").get_caller_identity().get("Account")

### 3.1 For preprocessing

In [47]:
%%writefile custom-docker/Dockerfile-prep

FROM 366743142698.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-scikit-learn:1.0-1-cpu-py3
RUN pip install -U pip
RUN pip install -U sagemaker
RUN pip3 install pyathena awswrangler 
ENV PYTHONUNBUFFERED=TRUE

Overwriting custom-docker/Dockerfile-prep


In [48]:
strRepositoryName="mlops-image-prep"  ## <-- 원하는 docker repostory 이름을 추가
strRepositoryName = strRepositoryName.lower()
strDockerFile = "Dockerfile-prep"
strDockerDir = "./custom-docker/"
strTag = "latest"

In [49]:
ecr.build_docker(strDockerDir, strDockerFile, strRepositoryName, strRegionName="ap-northeast-2", strAccountId="366743142698")

/home/ec2-user/SageMaker/mlops-step-alert/2.building-pipelines
/home/ec2-user/SageMaker/mlops-step-alert/2.building-pipelines/custom-docker
strDockerFile Dockerfile-prep
aws ecr get-login --region 'ap-northeast-2' --registry-ids '366743142698' --no-include-email


WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Login Succeeded

Sending build context to Docker daemon  7.168kB

Step 1/5 : FROM 366743142698.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-scikit-learn:1.0-1-cpu-py3
1.0-1-cpu-py3: Pulling from sagemaker-scikit-learn
68e7bb398b9f: Pulling fs layer
497a1f968dd9: Pulling fs layer
d97e19e8a6be: Pulling fs layer
cdd802750018: Pulling fs layer
0e8c1b3f12bc: Pulling fs layer
fa32be0c079f: Pulling fs layer
c566384c41d8: Pulling fs layer
e450908e073e: Pulling fs layer
e8a86a4af30f: Pulling fs layer
7dfba4ed7c4c: Pulling fs layer
3ca9131ed516: Pulling fs layer
045f5284d8f0: Pulling fs layer
fa32be0c079f: Waiting
f30007c75ed5: Pulling fs layer
d4f14c478a48: Pulling fs layer
87161d3a9f33: Pulling fs layer
31e5c31b9623: Pulling fs layer
40aa6dfd7717: Pulling fs layer
f892a107f29d: Pulling fs layer
e5907f94f874: Pulling fs layer
c566384c41d8: Waiting
cdd802750018: Waiting
e450908e073e: Waiting
e8a86a4af30f: Waiting
7dfba4ed7c4c: Waiting
d4f14c478a48: Waiting
3ca9131ed516: Waiting
87161d3a9f33: W

In [50]:
strEcrRepositoryUri = ecr.register_image_to_ecr(strRegionName, strAccountID, strRepositoryName, strTag)

== REGISTER AN IMAGE TO ECR ==
  processing_repository_uri: 419974056037.dkr.ecr.us-east-1.amazonaws.com/mlops-image-prep:latest
aws ecr get-login --region 'us-east-1' --registry-ids '419974056037' --no-include-email


WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Login Succeeded

aws ecr create-repository --repository-name 'mlops-image-prep'
docker tag 'mlops-image-prep:latest' '419974056037.dkr.ecr.us-east-1.amazonaws.com/mlops-image-prep:latest'
docker push '419974056037.dkr.ecr.us-east-1.amazonaws.com/mlops-image-prep:latest'
== REGISTER AN IMAGE TO ECR ==


### 3.2 For training

In [55]:
%%writefile custom-docker/Dockerfile-tr

FROM 366743142698.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-xgboost:1.5-1
RUN pip install -U pip
RUN pip install -U sagemaker
RUN pip install shap
RUN pip install sagemaker-inference
RUN pip install seaborn==0.11.2
ENV PATH="/opt/ml/code:${PATH}"
ENV SAGEMAKER_SUBMIT_DIRECTORY /opt/ml/code

# /opt/ml and all subdirectories are utilized by SageMaker, 
# use the /code subdirectory to store your user code.
#COPY train.py /opt/ml/code/train.py

# Defines train.py as script entrypoint
#ENV SAGEMAKER_PROGRAM train.py
ENV PYTHONUNBUFFERED=TRUE

Overwriting custom-docker/Dockerfile-tr


In [56]:
strTrRepositoryName="mlops-image-tr"  ## <-- 원하는 docker repostory 이름을 추가
strTrRepositoryName = strTrRepositoryName.lower()
strDockerFile = "Dockerfile-tr"
strDockerDir = "./custom-docker/"
strTag = "latest"

In [57]:
ecr.build_docker(strDockerDir, strDockerFile, strTrRepositoryName, strRegionName="ap-northeast-2", strAccountId="366743142698")

/home/ec2-user/SageMaker/mlops-step-alert/2.building-pipelines
/home/ec2-user/SageMaker/mlops-step-alert/2.building-pipelines/custom-docker
strDockerFile Dockerfile-tr
aws ecr get-login --region 'ap-northeast-2' --registry-ids '366743142698' --no-include-email


WARNING! Using --password via the CLI is insecure. Use --password-stdin.


Login Succeeded

Sending build context to Docker daemon  7.168kB

Step 1/9 : FROM 366743142698.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-xgboost:1.5-1
 ---> 2dee2c955721
Step 2/9 : RUN pip install -U pip
 ---> Using cache
 ---> 657d6c353448
Step 3/9 : RUN pip install -U sagemaker
 ---> Using cache
 ---> 6018b8f1f76f
Step 4/9 : RUN pip install shap
 ---> Using cache
 ---> dd58906f9e8c
Step 5/9 : RUN pip install sagemaker-inference
 ---> Using cache
 ---> dd416b9ce3a3
Step 6/9 : RUN pip install seaborn==0.11.2
 ---> Using cache
 ---> 250208a89bcb
Step 7/9 : ENV PATH="/opt/ml/code:${PATH}"
 ---> Using cache
 ---> 6d4f3d1f4ad9
Step 8/9 : ENV SAGEMAKER_SUBMIT_DIRECTORY /opt/ml/code
 ---> Using cache
 ---> e74595f60710
Step 9/9 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> 4686be875f2e
Successfully built 4686be875f2e
Successfully tagged mlops-image-tr:latest

/home/ec2-user/SageMaker/mlops-step-alert/2.building-pipelines


WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [58]:
strTrEcrRepositoryUri = ecr.register_image_to_ecr(strRegionName, strAccountID, strTrRepositoryName, strTag)

== REGISTER AN IMAGE TO ECR ==
  processing_repository_uri: 419974056037.dkr.ecr.us-east-1.amazonaws.com/mlops-image-tr:latest
aws ecr get-login --region 'us-east-1' --registry-ids '419974056037' --no-include-email


WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Login Succeeded

aws ecr create-repository --repository-name 'mlops-image-tr'
docker tag 'mlops-image-tr:latest' '419974056037.dkr.ecr.us-east-1.amazonaws.com/mlops-image-tr:latest'
docker push '419974056037.dkr.ecr.us-east-1.amazonaws.com/mlops-image-tr:latest'
== REGISTER AN IMAGE TO ECR ==


## 4. Set default parameters

In [59]:
pm.put_params(key="PREFIX", value=strPrefix, overwrite=True)
pm.put_params(key="-".join([strPrefix, "REGION"]), value=strRegionName, overwrite=True)
pm.put_params(key="-".join([strPrefix, "BUCKET"]), value=strBucketName, overwrite=True)
pm.put_params(key="-".join([strPrefix, "SAGEMAKER-ROLE-ARN"]), value=get_execution_role(), overwrite=True)
pm.put_params(key="-".join([strPrefix, "ACCOUNT-ID"]), value=strAccountID, overwrite=True)
pm.put_params(key="-".join([strPrefix, "IMAGE-URI-PREP"]), value=strEcrRepositoryUri, overwrite=True)
pm.put_params(key="-".join([strPrefix, "IMAGE-URI-TR"]), value=strTrEcrRepositoryUri, overwrite=True)
pm.put_params(key="-".join([strPrefix, "LAMBDA-ROLE-ARN"]), value=lambdaRoleArn, overwrite=True)
pm.put_params(key="-".join([strPrefix, "CODE-BUILD-ROLE-ARN"]), value=buildRoleArn, overwrite=True)
pm.put_params(key="-".join([strPrefix, "CODE-PIPELINE-ROLE-ARN"]), value=pipelineRoleArn, overwrite=True)
pm.put_params(key="-".join([strPrefix, "CODE-EVENTBRIDGE-ROLE-ARN"]), value=eventBridgeRoleArn, overwrite=True)

'Store suceess'

In [60]:
print ("PREFIX: ", pm.get_params(key="PREFIX"))
print ("REGION: ", pm.get_params(key="-".join([strPrefix, "REGION"])))
print ("BUCKET: ", pm.get_params(key="-".join([strPrefix, "BUCKET"])))
print ("SAGEMAKER-ROLE-ARN: ", pm.get_params(key="-".join([strPrefix, "SAGEMAKER-ROLE-ARN"])))
print ("ACCOUNT-ID: ", pm.get_params(key="-".join([strPrefix, "ACCOUNT-ID"])))
print ("IMAGE-URI-PREP: ", pm.get_params(key="-".join([strPrefix, "IMAGE-URI-PREP"])))
print ("IMAGE-URI-TR: ", pm.get_params(key="-".join([strPrefix, "IMAGE-URI-TR"])))
print ("LAMBDA-ROLE-ARN: ", pm.get_params(key="-".join([strPrefix, "LAMBDA-ROLE-ARN"])))
print ("CODE-BUILD-ROLE-ARN: ", pm.get_params(key="-".join([strPrefix, "CODE-BUILD-ROLE-ARN"])))
print ("CODE-PIPELINE-ROLE-ARN: ", pm.get_params(key="-".join([strPrefix, "CODE-PIPELINE-ROLE-ARN"])))
print ("CODE-EVENTBRIDGE-ROLE-ARN: ", pm.get_params(key="-".join([strPrefix, "CODE-EVENTBRIDGE-ROLE-ARN"])))
print ("DATA-PATH: ", pm.get_params(key="-".join([strPrefix, "DATA-PATH"])))

PREFIX:  DJ-SM-PIPELINE
REGION:  us-east-1
BUCKET:  sagemaker-us-east-1-419974056037
SAGEMAKER-ROLE-ARN:  arn:aws:iam::419974056037:role/service-role/AmazonSageMaker-ExecutionRole-20221206T163436
ACCOUNT-ID:  419974056037
IMAGE-URI-PREP:  419974056037.dkr.ecr.us-east-1.amazonaws.com/mlops-image-prep:latest
IMAGE-URI-TR:  419974056037.dkr.ecr.us-east-1.amazonaws.com/mlops-image-tr:latest
LAMBDA-ROLE-ARN:  arn:aws:iam::419974056037:role/DJ-SM-PIPELINE-LabmdaRole
CODE-BUILD-ROLE-ARN:  arn:aws:iam::419974056037:role/DJ-SM-PIPELINE-CodeBuildRole
CODE-PIPELINE-ROLE-ARN:  arn:aws:iam::419974056037:role/DJ-SM-PIPELINE-CodePipelineRole
CODE-EVENTBRIDGE-ROLE-ARN:  arn:aws:iam::419974056037:role/DJ-SM-PIPELINE-EventBridgeRole
DATA-PATH:  s3://sagemaker-us-east-1-419974056037/DJ-SM-PIPELINE-DATA


In [47]:
container = sagemaker.image_uris.retrieve("xgboost", strRegionName, "1.7-1")

In [48]:
container

'683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1'